# Notebook #4: Running Federated Training of the Pneumonia Model

### Install the Rhino Health Python SDK, Load All Necessary Libraries and Login to the Rhino FCP

In [ ]:
import getpass
import rhino_health as rh
from rhino_health.lib.endpoints.aimodel.aimodel_dataclass import (
    AIModelCreateInput,
    ModelTypes,
    AIModelRunInput,
    AIModelMultiCohortInput,
    AIModelTrainInput 
)

In [ ]:
my_username = "FCP_LOGIN_EMAIL" # Replace this with the email you use to log into Rhino Health
session = rh.login(username=my_username, password=getpass.getpass())

### Retrieve Project and Cohort Information

In [ ]:
project = session.project.get_project_by_name("YOUR_PROJECT_NAME")  # Replace with your project name

In [ ]:
# Get the schema that was created after JPG conversion
cxr_schema = project.get_data_schema_by_name('Auto-generated schema for mimic_cxr_hco_conv', project_uid=project.uid)
cxr_schema_uid =cxr_schema.uid
print(cxr_schema_uid)

In [ ]:
cohorts = project.cohorts
hco_cxr_cohort = project.get_cohort_by_name("mimic_cxr_hco_conv")
aidev_cxr_cohort = project.get_cohort_by_name("mimic_cxr_dev_conv")
cxr_cohorts = [aidev_cxr_cohort.uid, hco_cxr_cohort.uid]
print(f"Loaded CXR cohorts '{hco_cxr_cohort.uid}', '{aidev_cxr_cohort.uid}'")

## Create the Train Test Split Model and then Run it Over both CXR Cohorts
We will split both CXR Data Cohorts into two Cohorts one for training and the other testing
### We will use a Pre-defined Container Image with our Model

In [ ]:
train_split_image_uri = "913123821419.dkr.ecr.us-east-1.amazonaws.com/rhino-gc-workgroup-rhino-sandbox-decode-health:train-test-split-sb"

In [ ]:
aimodel = AIModelCreateInput(
    name="Train Test Split",
    description="Splitting data into train and test datasets per site",
    input_data_schema_uids=[cxr_schema_uid],
    output_data_schema_uids=[None], # Auto-Generating the Output Data Schema for the Model
    model_type=ModelTypes.GENERALIZED_COMPUTE,
    project_uid = project.uid,
    config={"container_image_uri": train_split_image_uri}
)
aimodel = session.aimodel.create_aimodel(aimodel)
print(f"Got aimodel '{aimodel.name}' with uid {aimodel.uid}")

run_params = AIModelMultiCohortInput(
    aimodel_uid= aimodel.uid,
    input_cohort_uids=[aidev_cxr_cohort.uid, hco_cxr_cohort.uid],
    output_cohort_naming_templates= ['{{ input_cohort_names.0 }} - Train', '{{ input_cohort_names.0 }} - Test'],
    timeout_seconds=600,
    sync=False,
)

print(f"Starting to run {aimodel.name}")
model_run = session.aimodel.run_aimodel(run_params)
run_result = model_run.wait_for_completion()
print(f"Finished running {aimodel.name}")
print(f"Result status is '{train_result.status.value}', errors={train_result.result_info.get('errors') if train_result.result_info else None}")

## Create and Run the Federated Model Training and Validation Across Both of Our Two Sites
We will utilize NVFlare to train our pneumonia predicition model using our local training Cohort and the remote Health System training Cohort.  The model will then be validated again the local testing Cohort and the remote Health System testing Cohort.
### We will use a Pre-defined Container Image with our Model

In [ ]:
model_train_image_uri = "913123821419.dkr.ecr.us-east-1.amazonaws.com/rhino-gc-workgroup-rhino-sandbox-decode-health:prediction-model-sb-22"

### Search for our Newly Split Local and Remote Cohorts 

In [ ]:
input_training_cohorts = session.cohort.search_for_cohorts_by_name('Train')
input_validation_cohorts =  session.cohort.search_for_cohorts_by_name('Test')
print("Found training cohorts:")
print([x.name for x in input_training_cohorts])
print("Found validation cohorts:")
print([x.name for x in input_validation_cohorts])

### Create the Pneumonia Prediciton Model

In [ ]:
aimodel = AIModelCreateInput(
    name="Pneumonia Prediction Model Training",
    description="Pneumonia Prediction Model Training",
    input_data_schema_uids=[cxr_schema_uid],
    output_data_schema_uids=[None], # Auto-Generating the Output Data Schema for the Model
    project_uid= project.uid,
    model_type=ModelTypes.NVIDIA_FLARE_V2_2,
    config={"container_image_uri": model_train_image_uri}
)

aimodel = session.aimodel.create_aimodel(aimodel)
print(f"Got aimodel '{aimodel.name}' with uid {aimodel.uid}")

### Run the Pneumonia Prediciton Model Training with Validation

In [ ]:
run_params = AIModelTrainInput(
    aimodel_uid=aimodel.uid,
    input_cohort_uids=[x.uid for x in input_training_cohorts], 
    one_fl_client_per_cohort=True ,        
    validation_cohort_uids=[x.uid for x in input_validation_cohorts], 
    validation_cohorts_inference_suffix=" - Pneumonia training results",
    timeout_seconds=600,
    config_fed_server="",
    config_fed_client="",
    secrets_fed_client="",
    secrets_fed_server="",
    sync=False,
)

print(f"Starting to run federated training of {aimodel.name}")
model_train = session.aimodel.train_aimodel(run_params)
train_result = model_train.wait_for_completion()
print(f"Finished running {aimodel.name}")
print(f"Result status is '{train_result.status.value}', errors={train_result.result_info.get('errors') if train_result.result_info else None}")